# ATR breakount - Analytics

### Import Library

In [17]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [18]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [19]:
data = dataset.copy()

In [20]:
EMA_LENGTH = 5
RSI_LENGTH = 20
# Calculate the EMA of Close
data["EMA_1"] = ta.ema(data['Close'], length=EMA_LENGTH)
# Calculate the EMA of EMA_1
data['EMA_2'] = ta.ema(data['EMA_1'], length=EMA_LENGTH)
# Calculate the EMA of EMA_2
data['EMA_3'] = ta.ema(data['EMA_2'], length=EMA_LENGTH)
data["RSI"] = ta.rsi(data["Close"], length=RSI_LENGTH)
def cal_tema(row):
    #TEMA = (3 x EMA1) – (3 x EMA2) + (EMA3)
    return (3*row['EMA_1']) - (3*row['EMA_2']) + row['EMA_3']
    
# Calculate the TEMA
data['TEMA'] = data.apply(lambda r: cal_tema(r), axis=1)
data.dropna(inplace=True)

In [21]:
data

,Open,High,Low,Close,Volume,EMA_1,EMA_2,EMA_3,RSI,TEMA
Date,,,,,,,,,,
2018-08-13 10:40:00,944.9,945.0,943.7,943.7,936,944.198319,944.456185,944.643842,48.074608,943.870246
2018-08-13 10:45:00,943.5,943.5,942.9,943.5,1793,943.965546,944.292639,944.526774,46.799059,943.545497
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,943.710364,944.098547,944.384032,44.917307,943.219483
2018-08-13 10:55:00,943.2,944.1,942.7,943.7,1236,943.706909,943.968001,944.245355,48.546932,943.462080
2018-08-13 11:00:00,943.7,944.2,943.3,944.2,1477,943.871273,943.935758,944.142156,51.884342,943.948700
...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,1270.345649,1266.978384,1264.553197,64.614686,1274.654993
2024-08-02 14:20:00,1274.6,1277.5,1274.5,1276.8,10914,1272.497100,1268.817956,1265.974783,67.090099,1277.012214
2024-08-02 14:25:00,1276.4,1277.4,1274.0,1276.5,11681,1273.831400,1270.489104,1267.479557,66.391679,1277.506444


### Calculate signal

In [22]:
#https://www.quantifiedstrategies.com/profitable-tema-trading-strategy/
#Go long when TEMA(5) > Closing Price and RSI(21) < 35%, hold the position until TEMA(5) < Closing Price and RSI(21) > 65%,
#Go short when TEMA(5) < Closing Price and RSI(21) > 65%, and hold the position until TEMA(5) > Closing Price and RSI(21) < 35%.
def cal_signal(row):
    signal = ''
    if row['TEMA'] > row['Close'] and row['RSI'] < 35:
        signal = 'long'
    elif row['TEMA'] < row['Close'] and row['RSI'] > 65:
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)

In [23]:
data

,Open,High,Low,Close,Volume,EMA_1,EMA_2,EMA_3,RSI,TEMA,signal
Date,,,,,,,,,,,
2018-08-13 10:40:00,944.9,945.0,943.7,943.7,936,944.198319,944.456185,944.643842,48.074608,943.870246,
2018-08-13 10:45:00,943.5,943.5,942.9,943.5,1793,943.965546,944.292639,944.526774,46.799059,943.545497,
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,943.710364,944.098547,944.384032,44.917307,943.219483,
2018-08-13 10:55:00,943.2,944.1,942.7,943.7,1236,943.706909,943.968001,944.245355,48.546932,943.462080,
2018-08-13 11:00:00,943.7,944.2,943.3,944.2,1477,943.871273,943.935758,944.142156,51.884342,943.948700,
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,1270.345649,1266.978384,1264.553197,64.614686,1274.654993,
2024-08-02 14:20:00,1274.6,1277.5,1274.5,1276.8,10914,1272.497100,1268.817956,1265.974783,67.090099,1277.012214,
2024-08-02 14:25:00,1276.4,1277.4,1274.0,1276.5,11681,1273.831400,1270.489104,1267.479557,66.391679,1277.506444,


In [24]:
from cal_return_v1 import cal_return
data = cal_return(data)
data

,Open,High,Low,Close,Volume,EMA_1,EMA_2,EMA_3,RSI,TEMA,signal,return,exit_time
Date,,,,,,,,,,,,,
2018-08-13 10:40:00,944.9,945.0,943.7,943.7,936,944.198319,944.456185,944.643842,48.074608,943.870246,,,
2018-08-13 10:45:00,943.5,943.5,942.9,943.5,1793,943.965546,944.292639,944.526774,46.799059,943.545497,,,
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,943.710364,944.098547,944.384032,44.917307,943.219483,,,
2018-08-13 10:55:00,943.2,944.1,942.7,943.7,1236,943.706909,943.968001,944.245355,48.546932,943.462080,,,
2018-08-13 11:00:00,943.7,944.2,943.3,944.2,1477,943.871273,943.935758,944.142156,51.884342,943.948700,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,1270.345649,1266.978384,1264.553197,64.614686,1274.654993,,,
2024-08-02 14:20:00,1274.6,1277.5,1274.5,1276.8,10914,1272.497100,1268.817956,1265.974783,67.090099,1277.012214,,,
2024-08-02 14:25:00,1276.4,1277.4,1274.0,1276.5,11681,1273.831400,1270.489104,1267.479557,66.391679,1277.506444,,,


In [25]:
has_return = data[data['return'] != '']

In [26]:
has_return['return'].sum()

-1576.1000000000008

In [27]:
len(has_return['return'])

7214

In [28]:
data[data.index > '2024-08-02 02:45:00']

,Open,High,Low,Close,Volume,EMA_1,EMA_2,EMA_3,RSI,TEMA,signal,return,exit_time
Date,,,,,,,,,,,,,
2024-08-02 09:00:00,1266.2,1266.7,1263.4,1264.6,11321,1272.912057,1276.944248,1279.921072,21.020762,1267.824500,long,,
2024-08-02 09:05:00,1264.3,1265.9,1264.3,1265.6,2145,1270.474705,1274.787734,1278.209959,22.844397,1265.270873,,,
2024-08-02 09:10:00,1265.5,1265.6,1264.6,1264.7,1777,1268.549803,1272.708424,1276.376114,22.355377,1263.900253,,,
2024-08-02 09:15:00,1264.8,1264.9,1263.2,1263.2,4721,1266.766536,1270.727794,1274.493341,21.546203,1262.609565,,,
2024-08-02 09:20:00,1263.4,1266.0,1262.3,1266.0,7022,1266.511024,1269.322204,1272.769628,26.755490,1264.336087,,,
2024-08-02 09:25:00,1266.0,1267.3,1264.7,1265.4,5954,1266.140682,1268.261697,1271.266985,26.360677,1264.903941,,,
2024-08-02 09:30:00,1265.5,1266.1,1264.3,1264.4,2505,1265.560455,1267.361283,1269.965084,25.695467,1264.562600,long,-3,2024-08-02 10:50:00
2024-08-02 09:35:00,1264.4,1264.8,1263.5,1263.5,2811,1264.873637,1266.532067,1268.820745,25.095513,1263.845453,long,-3,2024-08-02 10:50:00
2024-08-02 09:40:00,1263.6,1264.4,1263.5,1264.1,2060,1264.615758,1265.893298,1267.844929,26.303037,1264.012310,,,
